In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.paths import LOCAL_PROCESSED_DATA_PATH, LOCAL_RAW_DATA_PATH
from src.processing.dialogre_processing import DialogREDatasetTransformer
from src.processing.dataframe_utils import get_counts_and_percentages
from src.utils import handle_exceptions
import itertools
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

dt = DialogREDatasetTransformer(LOCAL_RAW_DATA_PATH / 'dialog-re')
df = dt.load_data_to_dataframe()
df


/home/ubuntu/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-07-20 14:17:29.328290: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/relnet/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .

,Dialogue,Relations,Origin
0,"[Speaker 1: Hey!, Speaker 2: Hey., Speaker 3: ...","[{'y': 'casting director', 'x': 'Ann', 'rid': ...",dev
1,"[Speaker 1, Speaker 2: Hi, Speaker 3: Hi! Hey ...","[{'y': 'Speaker 2', 'x': 'Speaker 1', 'rid': [...",dev
2,"[Speaker 1, Speaker 2: Hi!, Speaker 3: Hey!, S...","[{'y': 'man', 'x': 'Speaker 4', 'rid': [37], '...",dev
3,[Speaker 1: Wow! It looks like we got a lot of...,"[{'y': 'baby', 'x': 'Speaker 2', 'rid': [37], ...",dev
4,"[Speaker 1: Now, Mom, everything's going fine,...","[{'y': '26', 'x': 'Speaker 1', 'rid': [25], 'r...",dev
...,...,...,...
1783,"[Speaker 1: Nice camoflauge man, for a minute ...","[{'y': 'Speaker 1', 'x': 'Speaker 2', 'rid': [...",train
1784,"[Speaker 1: Well, I'm sure you'll teach her a ...","[{'y': 'Sir', 'x': 'Speaker 1', 'rid': [37], '...",train
1785,[Speaker 1: You know what? I can't even worry ...,"[{'y': 'baby', 'x': 'Speaker 1', 'rid': [37], ...",train
1786,"[Speaker 1: And cut. Hey, Butt Guy, what the h...","[{'y': 'Butt Guy', 'x': 'Speaker 2', 'rid': [3...",train


In [3]:
tmp = df.explode('Relations').reset_index()
col = ['index', 'x', 'y', 'r']
tmp = pd.concat([tmp, pd.json_normalize(tmp.Relations)],axis=1)
tmp['r'] = tmp['r'].apply(lambda x: x[0])
tmp = tmp[col]
tmp

,index,x,y,r
0,0,Ann,casting director,per:title
1,0,Ann,Annie,per:alternate_names
2,0,Estelle,agent,per:title
3,0,Estelle,Speaker 1,per:client
4,0,Speaker 2,Katelynn,per:alternate_names
...,...,...,...,...
9744,1787,Speaker 1,Speaker 2,per:acquaintance
9745,1787,Speaker 6,Pheebs,unanswerable
9746,1787,Speaker 6,Racquela,per:alternate_names
9747,1787,Speaker 2,Pheebs,per:alternate_names


In [4]:
tmp = df.explode('Relations').reset_index()
col = ['index', 'x', 'y', 'r']
tmp = pd.concat([tmp, pd.json_normalize(tmp.Relations)],axis=1)
tmp['r'] = tmp['r'].apply(lambda x: x[0])
tmp = tmp[col]
tmp

,index,x,y,r
0,0,Ann,casting director,per:title
1,0,Ann,Annie,per:alternate_names
2,0,Estelle,agent,per:title
3,0,Estelle,Speaker 1,per:client
4,0,Speaker 2,Katelynn,per:alternate_names
...,...,...,...,...
9744,1787,Speaker 1,Speaker 2,per:acquaintance
9745,1787,Speaker 6,Pheebs,unanswerable
9746,1787,Speaker 6,Racquela,per:alternate_names
9747,1787,Speaker 2,Pheebs,per:alternate_names


In [5]:
entity_sets = tmp.groupby('index', group_keys=True).apply(lambda row: set(list(row['x']) + list(row['y'])))
permutation_count = entity_sets.apply(lambda x: len(x)*(len(x)-1)).sum() # perhaps chekc if this is correct to compute the permutation count
print('expected amount of permutations =', permutation_count)
entity_sets

expected amount of permutations = 48534


index
0       {Speaker 1, Estelle, Annie, Ann, agent, Joey T...
1                 {Speaker 1, Jack, Speaker 3, Speaker 2}
2       {Speaker 1, Speaker 4, man, one, Geller, Speak...
3         {Speaker 1, Speaker 3, Speaker 2, baby, roomie}
4                                   {Speaker 1, Ross, 26}
                              ...                        
1783    {Speaker 1, man, Speaker 2, Chandler Bing, Sus...
1784               {sir, Speaker 1, Bing, Speaker 3, Sir}
1785      {Speaker 1, Rach, Speaker 2, little girl, baby}
1786                                {Butt Guy, Speaker 2}
1787    {Speaker 1, Pheebs, Speaker 2, Speaker 6, Racq...
Length: 1788, dtype: object

In [6]:
[(d['x'], d['y'], d['r'][0]) for d in df.iloc[0].Relations]

[('Ann', 'casting director', 'per:title'),
 ('Ann', 'Annie', 'per:alternate_names'),
 ('Estelle', 'agent', 'per:title'),
 ('Estelle', 'Speaker 1', 'per:client'),
 ('Speaker 2', 'Katelynn', 'per:alternate_names'),
 ('Speaker 2', 'Pheebs', 'per:alternate_names'),
 ('Speaker 2', 'Speaker 1', 'per:friends'),
 ('Speaker 2', 'Phoebe Buffay', 'per:alternate_names'),
 ('Annie', 'casting director', 'per:title'),
 ('Speaker 3', 'man', 'unanswerable'),
 ('Speaker 1', 'man', 'per:alternate_names'),
 ('Speaker 1', 'Speaker 2', 'per:friends'),
 ('Speaker 1', 'Joey Tribbiani', 'per:alternate_names')]

In [7]:
tmp2 = pd.DataFrame(entity_sets.apply(lambda x: [i for i in itertools.permutations(x, 2)]).explode(), columns=['entity_pairs'])
tmp2[['x', 'y']] = pd.DataFrame(tmp2['entity_pairs'].tolist(), index=tmp2.index)
tmp2 = tmp2[['x', 'y']]
tmp2

,x,y
index,,
0,Speaker 1,Estelle
0,Speaker 1,Annie
0,Speaker 1,Ann
0,Speaker 1,agent
0,Speaker 1,Joey Tribbiani
...,...,...
1787,Speaker 6,Racquela
1787,Racquela,Speaker 1
1787,Racquela,Pheebs


In [8]:
merged_df = tmp2.reset_index().merge(tmp, on=['index', 'x', 'y'], how='left')
merged_df.set_index('index', inplace=True)
merged_df

,x,y,r
index,,,
0,Speaker 1,Estelle,NaN
0,Speaker 1,Annie,NaN
0,Speaker 1,Ann,NaN
0,Speaker 1,agent,NaN
0,Speaker 1,Joey Tribbiani,per:alternate_names
...,...,...,...
1787,Speaker 6,Racquela,per:alternate_names
1787,Racquela,Speaker 1,NaN
1787,Racquela,Pheebs,NaN


In [9]:
tmp3 = tmp.rename({'x': 'y', 'y': 'x', 'r': 'r_swaped'}, axis=1)
tmp3

,index,y,x,r_swaped
0,0,Ann,casting director,per:title
1,0,Ann,Annie,per:alternate_names
2,0,Estelle,agent,per:title
3,0,Estelle,Speaker 1,per:client
4,0,Speaker 2,Katelynn,per:alternate_names
...,...,...,...,...
9744,1787,Speaker 1,Speaker 2,per:acquaintance
9745,1787,Speaker 6,Pheebs,unanswerable
9746,1787,Speaker 6,Racquela,per:alternate_names
9747,1787,Speaker 2,Pheebs,per:alternate_names


In [10]:
merged_df2 = merged_df.reset_index().merge(tmp3, on=['index', 'x', 'y'], how='left')
merged_df2.set_index('index', inplace=True)
merged_df2.loc[0]

,x,y,r,r_swaped
index,,,,
0,Speaker 1,Estelle,NaN,per:client
0,Speaker 1,Annie,NaN,NaN
0,Speaker 1,Ann,NaN,NaN
0,Speaker 1,agent,NaN,NaN
0,Speaker 1,Joey Tribbiani,per:alternate_names,NaN
...,...,...,...,...
0,Katelynn,Pheebs,NaN,NaN
0,Katelynn,Phoebe Buffay,NaN,NaN
0,Katelynn,casting director,NaN,NaN


In [11]:
import numpy as np

def extract_binary_relation(row):
    if row['r'] == 'unanswerable' or row['r_swaped'] == 'unanswerable' :
        return 'no_relation'
    elif pd.isnull(row['r']) and pd.isnull(row['r_swaped']):
        return 'no_relation'
    elif not pd.isnull(row['r']) and pd.isnull(row['r_swaped']):
        return 'with_relation'
    elif pd.isnull(row['r']) and not pd.isnull(row['r_swaped']):
        return 'with_relation'
    elif not pd.isnull(row['r']) and not pd.isnull(row['r_swaped']):
        return 'with_relation'
    else:
        return 'UNKNOWN'
    
merged_df2['r_binary'] = merged_df2.apply(extract_binary_relation, axis=1)

mask2 = ~merged_df2['r_swaped'].isna()
merged_df2[mask2].loc[0].sort_values(['x', 'y'])

,x,y,r,r_swaped,r_binary
index,,,,,
0,Annie,Ann,NaN,per:alternate_names,with_relation
0,Joey Tribbiani,Speaker 1,NaN,per:alternate_names,with_relation
0,Katelynn,Speaker 2,NaN,per:alternate_names,with_relation
0,Pheebs,Speaker 2,NaN,per:alternate_names,with_relation
0,Phoebe Buffay,Speaker 2,NaN,per:alternate_names,with_relation
0,Speaker 1,Estelle,NaN,per:client,with_relation
0,Speaker 1,Speaker 2,per:friends,per:friends,with_relation
0,Speaker 2,Speaker 1,per:friends,per:friends,with_relation
0,agent,Estelle,NaN,per:title,with_relation


In [12]:
import pandas as pd

# Assuming tmp is your DataFrame
tmp = merged_df2[['x', 'y', 'r_binary']].rename({'r_binary': 'r'}, axis=1).copy()
grouped = tmp.groupby('index')

symmetric = True
for name, group in grouped:
    for i in range(len(group)):
        row_i = group.iloc[i]
        x_i, y_i, r_i = row_i['x'], row_i['y'], row_i['r']
        subset = group[(group['x'] == y_i) & (group['y'] == x_i)]
        if not subset.empty:
            r_j = subset['r'].values[0]
            if r_i != r_j:
                symmetric = False
                print(f"Symmetry violation at index {name} between {x_i}, {y_i} and {y_i}, {x_i}. Relation is {r_i} and {r_j} respectively.")
                break
    if not symmetric:
        break

if symmetric:
    print("All relations are symmetric.")


All relations are symmetric.


In [13]:
import pandas as pd

# Assuming tmp is your DataFrame
tmp = merged_df2[['x', 'y', 'r_binary']].rename({'r_binary': 'r'}, axis=1).copy()
swapped = tmp.copy()
swapped.rename(columns={"x": "y", "y": "x"}, inplace=True)
merged = pd.merge(tmp, swapped, on=['index', 'x', 'y'], how='outer', suffixes=('', '_swapped'))

# Create mask
mask = merged['r'] != merged['r_swapped']

# Display all rows where 'r' values are not symmetric
non_symmetric_rows = merged[mask]

missing_relation_count = non_symmetric_rows.r_swapped.isna().sum()
print('missing_relation_count=',missing_relation_count)

non_symmetric_rows.sort_values(['index', 'x'])


missing_relation_count= 0


,x,y,r,r_swapped
index,,,,


In [14]:
final_df = merged_df2[['x', 'y', 'r_binary']].rename({'r_binary': 'r'}, axis=1).copy()
final_df = final_df.reset_index().drop_duplicates().set_index('index')
final_df

,x,y,r
index,,,
0,Speaker 1,Estelle,with_relation
0,Speaker 1,Annie,no_relation
0,Speaker 1,Ann,no_relation
0,Speaker 1,agent,no_relation
0,Speaker 1,Joey Tribbiani,with_relation
...,...,...,...
1787,Speaker 6,Racquela,with_relation
1787,Racquela,Speaker 1,no_relation
1787,Racquela,Pheebs,no_relation


In [15]:
final_dicts = final_df.groupby('index').apply(lambda x: x[['x','y','r']].to_dict('records'))
final_dicts

index
0       [{'x': 'Speaker 1', 'y': 'Estelle', 'r': 'with...
1       [{'x': 'Speaker 1', 'y': 'Jack', 'r': 'no_rela...
2       [{'x': 'Speaker 1', 'y': 'Speaker 4', 'r': 'no...
3       [{'x': 'Speaker 1', 'y': 'Speaker 3', 'r': 'no...
4       [{'x': 'Speaker 1', 'y': 'Ross', 'r': 'with_re...
                              ...                        
1783    [{'x': 'Speaker 1', 'y': 'man', 'r': 'no_relat...
1784    [{'x': 'sir', 'y': 'Speaker 1', 'r': 'no_relat...
1785    [{'x': 'Speaker 1', 'y': 'Rach', 'r': 'with_re...
1786    [{'x': 'Butt Guy', 'y': 'Speaker 2', 'r': 'wit...
1787    [{'x': 'Speaker 1', 'y': 'Pheebs', 'r': 'no_re...
Length: 1788, dtype: object

In [16]:
rid_dict = {"no_relation": 0, "with_relation": 1}
relations_df = df.Relations.explode().apply(pd.Series)[['x', 'y', 'x_type', 'y_type', 't']]
# Generate a type dictionary from x and y pairs
x_type_df = relations_df[['x', 'x_type']].drop_duplicates().rename(columns={'x': 'entity', 'x_type': 'type'})
y_type_df = relations_df[['y', 'y_type']].drop_duplicates().rename(columns={'y': 'entity', 'y_type': 'type'})
t_value_dict = relations_df.set_index(['x', 'y']).t.to_dict()

type_df = pd.concat([x_type_df, y_type_df]).drop_duplicates()

type_dict = type_df.set_index('entity').type.to_dict()

# Map the types to final_df
final_df['x_type'] = final_df['x'].map(type_dict)
final_df['y_type'] = final_df['y'].map(type_dict)

# Use this modified enrich_dicts function
def enrich_dicts(group):
    group_dicts = group[['x','y','r', 'x_type', 'y_type']].to_dict('records')

    for d in group_dicts:
        d['rid'] = [rid_dict.get(d['r'], 'UNKNOWN')]
        d['r'] = [d['r']]
        d['t'] = t_value_dict.get((d['x'], d['y']), [''])

    return group_dicts

final_dicts = final_df.groupby('index').apply(enrich_dicts)


In [17]:
df['Relations'] = final_dicts
df

,Dialogue,Relations,Origin
0,"[Speaker 1: Hey!, Speaker 2: Hey., Speaker 3: ...","[{'x': 'Speaker 1', 'y': 'Estelle', 'r': ['wit...",dev
1,"[Speaker 1, Speaker 2: Hi, Speaker 3: Hi! Hey ...","[{'x': 'Speaker 1', 'y': 'Jack', 'r': ['no_rel...",dev
2,"[Speaker 1, Speaker 2: Hi!, Speaker 3: Hey!, S...","[{'x': 'Speaker 1', 'y': 'Speaker 4', 'r': ['n...",dev
3,[Speaker 1: Wow! It looks like we got a lot of...,"[{'x': 'Speaker 1', 'y': 'Speaker 3', 'r': ['n...",dev
4,"[Speaker 1: Now, Mom, everything's going fine,...","[{'x': 'Speaker 1', 'y': 'Ross', 'r': ['with_r...",dev
...,...,...,...
1783,"[Speaker 1: Nice camoflauge man, for a minute ...","[{'x': 'Speaker 1', 'y': 'man', 'r': ['no_rela...",train
1784,"[Speaker 1: Well, I'm sure you'll teach her a ...","[{'x': 'sir', 'y': 'Speaker 1', 'r': ['no_rela...",train
1785,[Speaker 1: You know what? I can't even worry ...,"[{'x': 'Speaker 1', 'y': 'Rach', 'r': ['with_r...",train
1786,"[Speaker 1: And cut. Hey, Butt Guy, what the h...","[{'x': 'Butt Guy', 'y': 'Speaker 2', 'r': ['wi...",train


In [19]:
import json
import os
output_dir = LOCAL_PROCESSED_DATA_PATH / 'dialog-re-binary-validated'
os.makedirs(output_dir, exist_ok=True)

# Define your relation label dict
from collections import defaultdict

# Create a defaultdict
relation_label_dict = defaultdict(str)

# Iterate over the DataFrame
for relations in df['Relations']:
    # Iterate over the list of dictionaries
    for relation in relations:
        # Create an entry in the dictionary
        relation_label_dict[str(relation['rid'][0])] = relation['r'][0]

# Convert defaultdict back to dict
relation_label_dict = dict(relation_label_dict)


# Save to json
with open(output_dir / "relation_label_dict.json", "w") as f:
    json.dump(relation_label_dict, f)

formatted_data = df.apply(lambda row: [row['Dialogue'], row['Relations']], axis=1).tolist()

# Now save each subset to json files
data_splits = df['Origin'].unique()
for split in data_splits:
    subset_data = df[df['Origin'] == split].apply(lambda row: [row['Dialogue'], row['Relations']], axis=1).tolist()
    with open(output_dir / f'{split}.json', 'w') as f:
        json.dump(subset_data, f)


In [20]:
out = pd.DataFrame(pd.json_normalize(df.Relations.explode())['r'].str[0], columns=['r'])

In [21]:
get_counts_and_percentages(out, ['r'])

,Counts,%
r,,
no_relation,37332,76.9
with_relation,11202,23.1


In [25]:
dt = DialogREDatasetTransformer(LOCAL_PROCESSED_DATA_PATH / 'dialog-re-binary-validated-enriched')
df2 = dt.load_data_to_dataframe()
df2

,Dialogue,Relations,Origin
0,"[Speaker 1: Hey!, Speaker 2: Hey., Speaker 3: ...","[{'x': 'Speaker 1', 'y': 'Estelle', 'r': ['wit...",dev
1,"[Speaker 1, Speaker 2: Hi, Speaker 3: Hi! Hey ...","[{'x': 'Speaker 1', 'y': 'Jack', 'r': ['no_rel...",dev
2,"[Speaker 1, Speaker 2: Hi!, Speaker 3: Hey!, S...","[{'x': 'Speaker 1', 'y': 'Speaker 4', 'r': ['n...",dev
3,[Speaker 1: Wow! It looks like we got a lot of...,"[{'x': 'Speaker 1', 'y': 'Speaker 3', 'r': ['n...",dev
4,"[Speaker 1: Now, Mom, everything's going fine,...","[{'x': 'Speaker 1', 'y': 'Ross', 'r': ['with_r...",dev
...,...,...,...
1783,"[Speaker 1: Nice camoflauge man, for a minute ...","[{'x': 'Speaker 1', 'y': 'man', 'r': ['no_rela...",train
1784,"[Speaker 1: Well, I'm sure you'll teach her a ...","[{'x': 'sir', 'y': 'Speaker 1', 'r': ['no_rela...",train
1785,[Speaker 1: You know what? I can't even worry ...,"[{'x': 'Speaker 1', 'y': 'Rach', 'r': ['with_r...",train
1786,"[Speaker 1: And cut. Hey, Butt Guy, what the h...","[{'x': 'Butt Guy', 'y': 'Speaker 2', 'r': ['wi...",train


In [26]:
df2.Relations.explode().shape

(48534,)

0       {'x': 'Speaker 1', 'y': 'Estelle', 'r': ['with...
0       {'x': 'Speaker 1', 'y': 'Annie', 'r': ['no_rel...
0       {'x': 'Speaker 1', 'y': 'Ann', 'r': ['no_relat...
0       {'x': 'Speaker 1', 'y': 'agent', 'r': ['no_rel...
0       {'x': 'Speaker 1', 'y': 'Joey Tribbiani', 'r':...
                              ...                        
1787    {'x': 'Speaker 6', 'y': 'Racquela', 'r': ['wit...
1787    {'x': 'Racquela', 'y': 'Speaker 1', 'r': ['no_...
1787    {'x': 'Racquela', 'y': 'Pheebs', 'r': ['no_rel...
1787    {'x': 'Racquela', 'y': 'Speaker 2', 'r': ['no_...
1787    {'x': 'Racquela', 'y': 'Speaker 6', 'r': ['wit...
Name: Relations, Length: 48534, dtype: object

In [27]:
df2_reset = df2.reset_index()
exploded_df = df2_reset.explode('Relations')
tmp = pd.json_normalize(exploded_df['Relations'])
tmp['original_index'] = exploded_df['index'].values  # Use the index values from the exploded DataFrame
mask = tmp['min_words_distance'] < 0
tmp[mask][['x', 'y', 'r', 'original_index']]


KeyError: 'min_words_distance'

In [345]:
# 1770, 1777, 2, 1715
idx = 967
df.iloc[idx].Dialogue

["Speaker 1: Okay, well, this is crazy. Can't seriously be talking about me taking one of your kids, can we?",
 "Speaker 2: No, of course we're not.",
 'Speaker 1: Insane.',
 'Speaker 2: I know.',
 'Speaker 1: Alice would never go for it, right?',
 "Speaker 2: Oh, I don't know, she's pretty tired, too, I think we've got her onboard.",
 "Speaker 1: Well, just you know, for argument's sake, you know, hypothetically. Which one would you be willing to give up?",
 'Speaker 2: Huh.',
 'Speaker 1: Frank Jr.Jr.?',
 "Speaker 2: Oh, you'd be getting a really good one. I mean, you know, he's really funny. Like, the other day he made up this joke.",
 'Speaker 1: Uh-huh.',
 'Speaker 2: What\'s green and says "hey, I\'m a frog"? A talking frog! Oh, no, you can\'t have him, he\'s too funny.',
 "Speaker 1: Well, alright, that's fine. What about Leslie?",
 "Speaker 2: Oh, no, no. Not Leslie. No, she's, she's the only one that knows how to burp the alphabet.",
 'Speaker 1: Alright, so that leaves Chandl

In [343]:
df2.iloc[idx].Relations


[{'x': 'actor',
  'y': 'Pheebs',
  'r': ['no_relation'],
  'x_type': 'STRING',
  'y_type': 'PER',
  'rid': [0],
  't': [''],
  'x_token_span': [4, 5],
  'y_token_span': [57, 58],
  'x_char_span': [15, 21],
  'y_char_span': [216, 221],
  'min_words_distance': 54,
  'min_words_distance_pct': 0.12705882352941175,
  'spacy_features': {'x_pos': 'PROPN',
   'x_dep': 'dobj',
   'x_tag': 'NNP',
   'y_pos': 'NOUN',
   'y_dep': 'pobj',
   'y_tag': 'NN'},
  'min_turn_distance': 2,
  'min_turn_distance_pct': 0.004705882352941176},
 {'x': 'actor',
  'y': 'Pheebs',
  'r': ['no_relation'],
  'x_type': 'STRING',
  'y_type': 'PER',
  'rid': [0],
  't': [''],
  'x_token_span': [4, 5],
  'y_token_span': [57, 58],
  'x_char_span': [15, 21],
  'y_char_span': [216, 221],
  'min_words_distance': 54,
  'min_words_distance_pct': 0.12705882352941175,
  'spacy_features': {'x_pos': 'PROPN',
   'x_dep': 'dobj',
   'x_tag': 'NNP',
   'y_pos': 'NOUN',
   'y_dep': 'pobj',
   'y_tag': 'NN'},
  'min_turn_distance': 2,
